In [1]:
import torch 
import pickle
import math
import statistics
import numpy as np
import networkx as nx
from torch_geometric.loader import DataLoader
from torch_geometric.utils.convert import from_networkx
from torch_geometric.data import Dataset
from lib.utilities import Repository
import traceback



In [2]:
repo = Repository('./session_repositories/actions.tsv','./session_repositories/displays.tsv','./raw_datasets/')

with open(f'./edge/act_five_feats.pickle', 'rb') as fin:
    act_feats = pickle.load(fin)

with open(f'./edge/col_action.pickle', 'rb') as fin:
    col_feats = pickle.load(fin)

with open(f'./display_feats/display_pca_feats_{9999}.pickle', 'rb') as fin:
    display_pca_feats = pickle.load(fin)

num_col_classes = 0
actcol_feats = {}
for key in act_feats:
    num_col_classes = len(col_feats[key])
    feat = np.zeros(len(act_feats[key]) * len(col_feats[key]))
    offset = np.argmax(act_feats[key]) * len(col_feats[key])
    feat[offset + np.argmax(col_feats[key])] = 1
    actcol_feats[key] = feat.copy()

concat_feats = {}
for key in act_feats:
    concat_feats[key] = np.concatenate((act_feats[key], col_feats[key])).copy()


In [4]:
def get_predecessors(G, node):
    predecessors = []
    for v in G.neighbors(node):
        if v < node:
            predecessors.append(v)
    return predecessors

def get_successors(G, node):
    successors = []
    for v in G.neighbors(node):
        if v > node:
            successors.append(v)
    return successors

def bfs(G, root):
    depth = 0
    node_count_at_depth = {}
    nodes_at_depth = {}
    successors = [root]
    traversal = []
    while len(successors) > 0:
        # successors.sort()
        traversal.extend(successors)
        nodes_at_depth[depth] = []
        next_level = []
        for v in successors:
            nodes_at_depth[depth].append(v)
            next_level.extend(get_successors(G, v))

        node_count_at_depth[depth] = len(successors)
        depth += 1

        successors = next_level

    return traversal, node_count_at_depth, nodes_at_depth

def dfs(G, root):
    successors = get_successors(G, root)
    if len(successors) == 0:
        return [root]
    successors.sort()
    traversal = [root]
    for v in successors:
        v_traversal = dfs(G, v)
        traversal += v_traversal

    return traversal

def replay_graph(edges, d_feats, a_feats, tar, sizes, main_size, is_train):
    logic_error_displays = [427, 428, 429, 430, 
                        854, 855, 856, 868, 891, 
                        977, 978, 979, 980, 
                        1304, 1908, 1909, 1983, 
                        2022, 2023, 2024, 2195,
                        3244, 3446, 3447, 
                        4050, 4051, 4056, 4052, 4054, 4055, 4057, 4058, 4059, 
                        4060, 4061, 4062, 4063, 4064, 4065, 4066, 4067]

    replays = []
    display_seqs = []
    action_seqs = []
    y = []
    end_aids = []

    try:
        BigG = nx.from_edgelist(edges, create_using=nx.Graph)
        S = [BigG.subgraph(c).copy() for c in nx.connected_components(BigG)]

        # context = size
        for G in S:
            g_nodes = list(G.nodes())
            g_nodes.sort()
            g_nodes.reverse()
            for end in G.nodes():
                if end in logic_error_displays:
                    continue
                
                leading_to_end = None
                max_context = 0
                primary_nodes = None
                cxts = []
                for context in sizes:
                    tree_nodes = set()
                    for v in g_nodes:
                        if v < end and len(tree_nodes) < context:
                            tree_nodes.add(v)

                    if len(tree_nodes) > max_context:
                        max_context = len(tree_nodes)
                        primary_nodes = list(tree_nodes)

                    if len(tree_nodes) < context:
                        break

                    leading_to_end = get_predecessors(G, end)[0]
                    
                    tree_nodes_list = list(tree_nodes)
                    backtracks = []
                    max_path_length = 0
                    max_path_index = 0
                    for j in range(len(tree_nodes)):
                        v = tree_nodes_list[j]
                        predecessors = get_predecessors(G, v)
                        path = [v]
                        while len(predecessors) > 0:
                            path.append(predecessors[0])
                            predecessors = get_predecessors(G, predecessors[0])
                        
                        if len(path) > max_path_length:
                            max_path_length = len(path)
                            max_path_index = j
                        
                        path.reverse()
                        backtracks.append(path)
                    
                    proceed = True
                    i = -1
                    while proceed:
                        i += 1
                        if i >= max_path_length:
                            break

                        match_value = backtracks[max_path_index][i]
                        for path in backtracks:
                            if len(path) <= i:
                                proceed = False
                                break
                            if match_value != path[i]:
                                proceed = False
                                break
                        
                    for path in backtracks:
                        for v in path[i-1:]:
                            tree_nodes.add(v)

                    g_context = nx.induced_subgraph(G, tree_nodes).copy()

                    root = None
                    for v in g_context.nodes():
                        if len(get_predecessors(g_context, v)) == 0:
                            root = v

                    node_order, node_count_at_depth, nodes_at_depth = bfs(g_context, root)

                    node_depth = {}
                    for depth in nodes_at_depth:
                        depth_nodes = nodes_at_depth[depth]
                        depth_nodes.sort()
                        for i in range(len(depth_nodes)):
                            v = depth_nodes[i]
                            node_depth[v] = (depth, i)

                    tree_nodes = list(tree_nodes)
                    tree_nodes.sort()
                    attrs = {}
                    for i in range(len(tree_nodes)):
                        v = tree_nodes[i]
                        v_feat = d_feats[v]
                        attrs[v] = {"x":v_feat.astype(np.float32), "pos":node_depth[v]}
                    nx.set_node_attributes(g_context, attrs)

                    attrs = {}
                    for edge in g_context.edges():
                        e_feat = a_feats[g_context.edges[edge[0], edge[1]]['aid']]
                        attrs[(edge[0], edge[1])] = {"x":e_feat.astype(np.float32)}
                    nx.set_edge_attributes(g_context, attrs)

                    cxts.append(g_context)

                if len(cxts) == len(sizes):
                    end_aid = G.edges[leading_to_end, end]['aid']
                    replays.append(cxts)
                    end_aids.append(end_aid)

                    if is_train:
                        target_act = tar[0][end_aid]
                        target_col = tar[1][end_aid]
                        target_tg = tar[2][end_aid]
                    else:
                        target_act = np.argmax(tar[0][end_aid])
                        target_col = np.argmax(tar[1][end_aid])
                        target_tg = np.argmax(tar[2][end_aid])
                    y.append([target_act, target_col, target_tg])

                    ### SEQUENCE STUFF ###
                    assert max_context == main_size
                    primary_nodes.sort()
                    main_g_context = cxts[0]
                    action_sequence = []
                    for v in primary_nodes[1:]:
                        u = get_predecessors(main_g_context, v)[0]
                        action_sequence.append(main_g_context.edges[u, v]['x'])
                    
                    display_sequence = []
                    for v in primary_nodes:
                        display_sequence.append(main_g_context.nodes[v]['x'])
                    
                    action_seqs.append(torch.tensor(np.array(action_sequence), dtype=torch.float32))
                    display_seqs.append(torch.tensor(np.array(display_sequence), dtype=torch.float32))
                
    except Exception as e:
        # print(g_context.edges.data())
        # print(traceback.format_exc())
        print()

    return replays, action_seqs, display_seqs, y, end_aids


def generate_sessions(repo):
    og_columns = ['captured_length', 'length', 'tcp_stream', 'number', 'eth_dst', 'eth_src', 
                'highest_layer', 'info_line', 'interface_captured', 'ip_dst', 'ip_src', 'sniff_timestamp', 'tcp_dstport', 'tcp_srcport']
    sessions = {}
    for project_id in range(1, 5):
        my_sessions = []
        for session_id in repo.actions[repo.actions['project_id'] == project_id]['session_id'].unique():
            nodes = set()
            edges = []
            unrelated = False
            for i, row in repo.actions[repo.actions['session_id'] == session_id][['action_id', 'action_type', 'action_params', 'parent_display_id', 'child_display_id', 'solution']].iterrows():
                solution = 1 if row['solution'] else 0
                u = int(row['parent_display_id'])
                v = int(row['child_display_id'])
                aid = int(row['action_id'])

                nodes.add(u)
                nodes.add(v)

                if row['action_type'] == 'sort' and (not bool(row['action_params'])):
                    check_col = 'number'
                    row['action_params']['field'] = 'number'
                else:
                    check_col = row['action_params']['field']

                if not check_col in og_columns:
                    unrelated = True

                edges.append([u, v, {'aid':aid}])

            if not unrelated:
                my_sessions.append(edges)

        sessions[project_id] = my_sessions
    
    return sessions


def treefy_sessions(sessions, d_feats, a_feats, tar, sizes, main_size, test_id):
    test_contexts = []
    test_act_seqs = []
    test_display_seqs = []
    test_y = []
    test_aids = []
    train_contexts = []
    train_act_seqs = []
    train_display_seqs = []
    train_y = []
    train_aids = []
    for chunk_id in sessions:
        if chunk_id in test_id:
            for edges in sessions[chunk_id]:
                g_contexts, act_seqs, display_seqs, g_ys, end_aids = replay_graph(edges, d_feats, a_feats, tar, sizes=sizes, main_size=main_size, is_train=False)
                test_contexts.extend(g_contexts)
                test_act_seqs.extend(act_seqs)
                test_display_seqs.extend(display_seqs)
                test_y.extend(g_ys)
                test_aids.extend(end_aids)
        else:
            for edges in sessions[chunk_id]:
                g_contexts, act_seqs, display_seqs, g_ys, end_aids = replay_graph(edges, d_feats, a_feats, tar, sizes=sizes, main_size=main_size, is_train=True)
                train_contexts.extend(g_contexts)
                train_act_seqs.extend(act_seqs)
                train_display_seqs.extend(display_seqs)
                train_y.extend(g_ys)
                train_aids.extend(end_aids)

    return train_contexts, train_act_seqs, train_display_seqs, train_y, train_aids, test_contexts, test_act_seqs, test_display_seqs, test_y, test_aids



In [5]:
class SeqTestDataset(Dataset):
    def __init__(self, seqs, lumps, ys):
        super(SeqTestDataset, self).__init__()
        self.seqs = seqs
        self.lumps = lumps
        self.ys = ys

    def len(self):
        return len(self.seqs)
    
    def get(self, idx):
        seq = self.seqs[idx]
        lump = self.lumps[idx]
        return seq, lump, torch.tensor(self.ys[idx], dtype=torch.long)

def strip_graph_attributes(graph):
    for (_, d) in graph.nodes(data=True):
        del d['pos']
    for (_, _, d) in graph.edges(data=True):
        del d['aid']

def class_seperation(y):
    classes = {}
    for c in set(y):
        classes[c] = []

    for i in range(len(y)):
        classes[y[i]].append(i)

    return classes
    
def make_directed(graph_sets):
    directed_graphs = []
    for graphs in graph_sets:
        curr_set = []
        for i in range(len(graphs)):
            graph = graphs[i]
            if graph is None:
                curr_set.append(None)
            else:
                strip_graph_attributes(graph)
                dg = graph.to_directed()
                to_remove = []
                for edge in dg.edges():
                    if edge[0] > edge[1]:
                        to_remove.append(edge)

                dg.remove_edges_from(to_remove)
                pyg = from_networkx(dg)
                if graph.number_of_nodes() == 1:
                    pyg.edge_x = torch.empty((0, 20), dtype=torch.float)
                
                curr_set.append(pyg)
                
        directed_graphs.append(curr_set)

    return directed_graphs

def generate_lump_graphs(graph_sets, classes):
    lump_graphs = {}
    for c in classes:
        pos_feats = {}
        for i in classes[c]:
            for graph in graph_sets[i]:
                if graph is None:
                    continue
                for node in graph.nodes():
                    pos = graph.nodes[node]['pos']
                    if not (pos[0] in pos_feats):
                        pos_feats[pos[0]] = {}
                    if not (pos[1] in pos_feats[pos[0]]):
                        pos_feats[pos[0]][pos[1]] = []
                    pos_feats[pos[0]][pos[1]].append(graph.nodes[node]['x'])

        pos_to_id_map = {}
        pos_id = 0
        for depth in pos_feats:
            pos_to_id_map[depth] = {}
            for order in pos_feats[depth]:
                pos_feats[depth][order] = np.array(pos_feats[depth][order], dtype=np.float32).mean(axis=0)
                pos_to_id_map[depth][order] = pos_id
                pos_id += 1

        pos_edge_feats = {}
        for i in classes[c]:
            for graph in graph_sets[i]:
                if graph is None:
                    continue
                for edge in graph.edges():
                    u = min(edge[0], edge[1])
                    v = max(edge[0], edge[1])
                    u_pos = graph.nodes[u]['pos']
                    v_pos = graph.nodes[v]['pos']
                    u_id = pos_to_id_map[u_pos[0]][u_pos[1]]
                    v_id = pos_to_id_map[v_pos[0]][v_pos[1]]
                    
                    if not (u_id in pos_edge_feats):
                        pos_edge_feats[u_id] = {}
                    if not (v_id in pos_edge_feats[u_id]):
                        pos_edge_feats[u_id][v_id] = []

                    pos_edge_feats[u_id][v_id].append(graph.edges[edge[0], edge[1]]['x'])
        
        lump_graph_edges = []
        for u_id in pos_edge_feats:
            for v_id in pos_edge_feats[u_id]:
                pos_edge_feats[u_id][v_id] = np.array(pos_edge_feats[u_id][v_id], dtype=np.float32).mean(axis=0)
                lump_graph_edges.append((u_id, v_id, {"x":pos_edge_feats[u_id][v_id]}))

        node_attrs = {}
        for depth in pos_feats:
            for order in pos_feats[depth]:
                node_attrs[pos_to_id_map[depth][order]] = {"x":pos_feats[depth][order]}
                # node_attrs[pos_to_id_map[depth][order]] = {"x":pos_feats[depth][order], "pos":(depth, order)}
        
        lump_g = nx.from_edgelist(lump_graph_edges, create_using=nx.DiGraph)
        nx.set_node_attributes(lump_g, node_attrs)

        lump_graphs[c] = lump_g

    return lump_graphs

def generate_test_pairs(graph_set, y, lump_graphs):
    graph_sets_new = []
    lumps = []
    labels = []
    for c in lump_graphs:
        # graph_set.reverse()
        graph_sets_new.append(graph_set)
        lumps.append(from_networkx(lump_graphs[c]))
        labels.append(c)
    
    return graph_sets_new, lumps, labels

In [6]:
def apply_naive(act_probs, col_probs, tg_probs, classes, test_y, k):
    preds = []
    for m in range(len(test_y)):
        test_case = [0] * len(tg_probs[m])
        for i, act_prob in enumerate(act_probs[m]):
            act_cls = classes['act'][i]
            for j, col_prob in enumerate(col_probs[m]):
                col_cls = classes['col'][j]
                actcol_mass = act_prob * col_prob
                for n, tg_mass in enumerate(tg_probs[m]):
                    tgd_cls = classes['tgd'][n]
                    tgd_act, tgd_col = tgd_cls[0], tgd_cls[1]
                    if act_cls == tgd_act and col_cls == tgd_col:
                        test_case[n] += actcol_mass * tg_mass

        indices = torch.topk(torch.tensor(test_case, dtype=torch.float32), k, dim=0).indices.tolist()
        preds.append(classes['tg'][indices].tolist())
        
    corrects = [0] * len(test_y)
    mrrs = [0] * len(test_y)
    total = 0
    for i in range(len(test_y)):
        total += 1
        for j in range(len(preds[i])):
            if test_y[i][2] == preds[i][j]:
                corrects[i] = 1
                mrrs[i] = 1 / (j + 1)
    
    correct = sum(corrects)
    mrr = sum(mrrs)
    acc = round(correct / total, 4)
    mrr_acc = round(mrr / total, 4) 

    return acc, mrr_acc

In [7]:
def apply_dst(act_probs, col_probs, tg_probs, classes, test_y, k):
    preds = []
    for m in range(len(test_y)):
        test_case = [0] * len(tg_probs[m])
        K = 0
        for i, act_prob in enumerate(act_probs[m]):
            act_cls = classes['act'][i]
            for j, col_prob in enumerate(col_probs[m]):
                col_cls = classes['col'][j]
                actcol_mass = act_prob * col_prob
                for n, tg_mass in enumerate(tg_probs[m]):
                    tgd_cls = classes['tgd'][n]
                    tgd_act, tgd_col = tgd_cls[0], tgd_cls[1]
                    if act_cls != tgd_act or col_cls != tgd_col:
                        K += actcol_mass * tg_mass
                    else:
                        test_case[n] += actcol_mass * tg_mass

        for n in range(len(test_case)):
            test_case[n] = test_case[n] / (1 - K)

        indices = torch.topk(torch.tensor(test_case, dtype=torch.float32), k, dim=0).indices.tolist()
        preds.append(classes['tg'][indices].tolist())
        
    corrects = [0] * len(test_y)
    mrrs = [0] * len(test_y)
    total = 0
    for i in range(len(test_y)):
        total += 1
        for j in range(len(preds[i])):
            if test_y[i][2] == preds[i][j]:
                corrects[i] = 1
                mrrs[i] = 1 / (j + 1)
    
    correct = sum(corrects)
    mrr = sum(mrrs)
    acc = round(correct / total, 4)
    mrr_acc = round(mrr / total, 4) 

    return acc, mrr_acc

In [ ]:
# seed = 20250212
# test_id = [4]

file_fix = "own" # 'own'=EF-MP | 'seq'=EF-SP

tar_feats = [act_feats, col_feats, actcol_feats]

ra3_gains_react = []
mrr_gains_react = []
ra3_gains_noef = []
mrr_gains_noef = []

entry = "&"
gain_entry = "&"

for main_size in range(3, 9):
    sizes = list(range(1, main_size + 1))
    sizes.reverse()

    overall_ra3 = {'react':[], 'ours':[], 'dst':[]}
    overall_mrr = {'react':[], 'ours':[], 'dst':[]}

    for seed in [20250212, 20250214, 20250314]:
        chunked_sessions = pickle.load(open(f'./chunked_sessions/unbiased_seed_{seed}.pickle', 'rb'))
        for tid in range(5):
            test_id = [tid]

            train_x, train_act_seqs, train_display_seqs, train_y, train_aids, test_x, test_act_seqs, test_display_seqs, test_y, test_aids = treefy_sessions(
                                                                                                                                                sessions=chunked_sessions, 
                                                                                                                                                d_feats=display_pca_feats, 
                                                                                                                                                a_feats=concat_feats,
                                                                                                                                                tar=tar_feats, 
                                                                                                                                                sizes=sizes, 
                                                                                                                                                main_size=main_size,
                                                                                                                                                test_id=test_id
                                                                                                                                            )
            # print(len(train_y), len(test_y))

            lump_graphs = {}
            for j, task in enumerate(['act', 'col', 'tg']):
                non_hot_train_y = [int(np.argmax(train_y[i][j])) for i in range(len(train_y))]
                train_classes = class_seperation(non_hot_train_y)
                lump_graphs[task] = generate_lump_graphs(train_x, train_classes)

            train_x = make_directed(train_x)
            test_x = make_directed(test_x)

            classes = {}
            for task in ['act', 'col', 'tg']:
                clss = []
                for c in lump_graphs[task]:
                    clss.append(c)
                classes[task] = clss

            clss = []
            for c in classes['tg']:
                clss.append([math.floor(c / num_col_classes), c % num_col_classes])
            classes['tgd'] = clss
            classes['tg'] = torch.tensor(classes['tg'], dtype=torch.long)

            act_prob = pickle.load(open(f'./dst_probs/gine_{file_fix}_act_best_ra3_{seed}_{test_id}_{main_size}.pickle', 'rb'))
            col_prob = pickle.load(open(f'./dst_probs/gine_{file_fix}_col_best_ra3_{seed}_{test_id}_{main_size}.pickle', 'rb'))
            tg_prob = pickle.load(open(f'./dst_probs/gine_{file_fix}_tg_best_ra3_{seed}_{test_id}_{main_size}.pickle', 'rb'))

            # dst_ra3, dst_mrr = apply_dst(act_prob, col_prob, tg_prob, classes, test_y, 3)
            dst_ra3, dst_mrr = apply_naive(act_prob, col_prob, tg_prob, classes, test_y, 3)
            overall_ra3['dst'].append(dst_ra3)
            overall_mrr['dst'].append(dst_mrr)

            res = pickle.load(open(f'./chunk_ted_results/{seed}_{main_size}_{tid}_unbiased.pickle', 'rb'))
            overall_ra3['react'].append(res['ra3'][2])
            overall_mrr['react'].append(res['mrr'][2])

            res = pickle.load(open(f'./model_stats/tg_{seed}_{main_size}_[{tid}]_gine_{file_fix}.pickle', 'rb'))
            overall_ra3['ours'].extend(res['ra3'])
            overall_mrr['ours'].extend(res['mrr'])

    react_ra3, ours_ra3, dst_ra3 = statistics.mean(overall_ra3['react']), statistics.mean(overall_ra3['ours']), statistics.mean(overall_ra3['dst'])
    ra3_gain_react = round(((dst_ra3 - react_ra3) / react_ra3) * 100, 2)
    ra3_gain_noef = round(((dst_ra3 - ours_ra3) / ours_ra3) * 100, 2)
    react_mrr, ours_mrr, dst_mrr = statistics.mean(overall_mrr['react']), statistics.mean(overall_mrr['ours']), statistics.mean(overall_mrr['dst'])
    mrr_gain_react = round(((dst_mrr - react_mrr) / react_mrr) * 100, 2)
    mrr_gain_noef = round(((dst_mrr - ours_mrr) / ours_mrr) * 100, 2)

    ra3_gains_react.append(ra3_gain_react)
    mrr_gains_react.append(mrr_gain_react)

    ra3_gains_noef.append(ra3_gain_noef)
    mrr_gains_noef.append(mrr_gain_noef)

    # print(f'------------------{seed}-{main_size}-------------------')
    # print(round(react_ra3, 2), round(ours_ra3, 2), round(dst_ra3, 2), f'{ra3_gain}%')
    # print(round(react_mrr, 2), round(ours_mrr, 2), round(dst_mrr, 2), f'{mrr_gain}%')

    dst_ra3 = int(round(dst_ra3, 2) * 100)
    dst_mrr = int(round(dst_mrr, 2) * 100)

    ra3_stdv = str(round(statistics.stdev(overall_ra3['dst']), 2)).lstrip("0")
    mrr_stdv = str(round(statistics.stdev(overall_mrr['dst']), 2)).lstrip("0")

    entry += f" \\begin{{tabular}}[c]{{@{{}}l@{{}}}}$.{dst_ra3}_{{{ra3_stdv}}}$\\\\ $.{dst_mrr}_{{{mrr_stdv}}}$\end{{tabular}} &\n"
    # entry += f" \\begin{{tabular}}[c]{{@{{}}l@{{}}}}$\\mathbf{{.{dst_ra3}}}_{{{ra3_stdv}}}$\\\\ $\\mathbf{{.{dst_mrr}}}_{{{mrr_stdv}}}$\end{{tabular}} &\n"

    gain_entry += f" \\begin{{tabular}}[c]{{@{{}}l@{{}}}}${ra3_gain_react}\\%$\\\\ ${mrr_gain_react}\\%$\end{{tabular}} &\n"

print(entry)
print(f"Improv. over REACT - Min={min(ra3_gains_react)}-R@3, {min(mrr_gains_react)}-MRR | Max={max(ra3_gains_react)}-R@3, {max(mrr_gains_react)}-MRR | Avg={statistics.mean(ra3_gains_react)}-R@3, {statistics.mean(mrr_gains_react)}-MRR")
print(f"Improv. over no EF - Min={min(ra3_gains_noef)}-R@3, {min(mrr_gains_noef)}-MRR | Max={max(ra3_gains_noef)}-R@3, {max(mrr_gains_noef)}-MRR")
print()
print(gain_entry)





















































































































































































& \begin{tabular}[c]{@{}l@{}}$.48_{.04}$\\ $.33_{.03}$\end{tabular} &
 \begin{tabular}[c]{@{}l@{}}$.44_{.04}$\\ $.31_{.03}$\end{tabular} &
 \begin{tabular}[c]{@{}l@{}}$.43_{.06}$\\ $.30_{.03}$\end{tabular} &
 \begin{tabular}[c]{@{}l@{}}$.42_{.06}$\\ $.28_{.04}$\end{tabular} &
 \begin{tabular}[c]{@{}l@{}}$.39_{.07}$\\ $.27_{.04}$\end{tabular} &
 \begin{tabular}[c]{@{}l@{}}$.40_{.04}$\\ $.27_{.03}$\end{tabular} &

Min=7.72, 8.54 | Max=20.94, 13.87 Avg=13.691666666666666, 11.548333333333332
Min=10.62, 7.52 Max=20.73, 31.53

& \begin{tabular}[c]{@{}l@{}}$12.95\%$\\ $11.51\%$\end{tabular} &
 \begin{tabular}[c]{@{}l@{}}$10.88\%$\\ $13.2\%$\end{tabular} &
 \begin{tabular}[c]{@{}l@{}}$7.72\%$\\ $9.13\%$\end{tabular} &
 \begin{tabular}[c]{@{}l@{}}$12.9\%$\\ $8.54\%$\end{tabular} &
 \begin{tabular}[c]{@{}l@{}}$20.94\%$